# cr3 CTF 2024
##  guessing | 286 pts

Task description:

```
You send a matrix, then you get a new one. As simple as that.

$ nc 1337.sb 20000
```

Attachments: [main.sage](./main.sage)

The most interesting part is this function:

In [44]:
def submit_b(B):
    if not B.is_invertible():
        return "non invertible"

    n = B.nrows()
    if n != FLAG_N:
        return "rows"

    diagonal_element = B[0, 0]

    identity_multiple = Matrix(F, identity_matrix(n))
    for i in range(n):
        for j in range(n):
            identity_multiple[i, j] = diagonal_element

    if B == identity_multiple:
        return "identity_multiple"

    return B * A * (B ^ -1) * (A ^ -1) * FLAG

print("aight, lets see what you got")
while True:
    try:
        matrix = deserialize_mtx(input())
        res = submit_b(matrix)
        if isinstance(res, str):
            print("https://youtu.be/G7b4VMEATNo?t=18")
            break
    except Exception as E:
        print(E)
        print("https://youtu.be/G7b4VMEATNo?t=19")
        break

    print(serialize_mtx(res))

## Solution

Here we need to provide server with a matrix $B$ and somehow recover flag. There's a check for $B$ being scalar matrix, however this challenge had a bug, and you are able to send identity matrix, $A$ annihilate itself and we get the flag.

In [45]:
p = (2 ** 31) - 1
F = GF(p)
FLAG_N = 55

def serialize_mtx(mtx):
    rows = []
    for row in mtx.rows():
        rows.append(','.join(str(elem) for elem in row))
    return '[' + ';'.join(rows) + ']'


def deserialize_mtx(mtx):
    print(mtx)
    rows = mtx.strip('[]\n').split(';')
    rows = [[int(num) for num in row.split(',')] for row in rows]
    return Matrix(F, rows)

In [50]:
from pwn import remote, context
context.log_level = "error"

host, port = "1337.sb",  20000
r = remote(host,port)

# handle pow
print(r.recvline()) 
print(r.recvline())
r.sendline(input().encode())

mtx = identity_matrix(FLAG_N)
ser = serialize_mtx(mtx)

print(r.recvline())
r.sendline(ser.encode())
res = r.recvline().decode()
r.close()

bytes(list(deserialize_mtx(res).T[0]))

b'proof of work:\n'
b'curl -sSfL https://pwn.red/pow | sh -s s.AAATiA==.0FDexwacoWQpkRp9321qlA==\n'


 s.DPMAH595EdWby5rmB3PEisKgcRwTCVcat1sZZzZRSWzUvCkcaHl10855hVAJAZbnEXglsVai791V8j7WxaTwMkbHqK0YqBt/Hz7xa/hPk4ZHU6x+ahCGTXAxxvWdJlRPuytxUdDg3bsylTK5swi+vjYO0aMsAqD+97DUduqnktGQVz2MIAlqiZfxUQEXiFOPWS8gYomuxUEk5t8ZIyRysw==


b'solution: aight, lets see what you got\n'
[99;114;51;123;49;95;104;48;112;51;95;121;48;117;95;119;51;114;101;110;116;95;116;82;121;49;110;71;95;116;48;95;102;49;78;68;95;99;48;77;109;85;116;49;110;103;95;109;52;116;114;49;120;33;125]



b'cr3{1_h0p3_y0u_w3rent_tRy1nG_t0_f1ND_c0MmUt1ng_m4tr1x!}'